In [ ]:
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import json
import seaborn as sns

In [ ]:
def read_traces_file(fname):
    df = pd.read_csv(fname,sep = '\t',
           comment = '#', names = [x for x in range(10)])
    return df

def extract_metadata(fname):
    with open(fname) as f:
        dictio = json.load(f)
        r_height = dictio['map_info']['height']
        r_width = dictio['map_info']['width']
        
        return float(r_height), float(r_width)

def extract_traces(fname):
    traces = []
    full_df = read_traces_file(fname)
    wayp = full_df[full_df[1]== 'TYPE_WAYPOINT']
    coor = wayp.iloc[:,2:4]
    for i in range(coor.shape[0]):
        traces.append((float(coor.iloc[i][2]),float(coor.iloc[i][3])))
    return traces
        
def draw_map(map_image,metadata_fname, traces_fname, arrow = False):
    
    """
    This function can draw the locations with points of several colors from a list and one is chosen randomly.
    This function can also draw an arrow from the previous to the current poiny if chosen (the arrow is green)
    """
    
    colors = [(255,0,0), (0,0,255), (255,0,255), (128,0,0),
             (0,128,128), (0,255,255), (250,235,215),(205,133,63),
             (112,128,144), (230,230,250),(240,255,255), (255,105,180),
             (128,0,128), (30,144,255)] # this is the color pallet
    color = colors[np.random.randint(len(colors), size = 1)[0]]
    
    img = map_image
    traces = extract_traces(traces_fname)
    real_height, real_width = extract_metadata(metadata_fname)
    img_height = img.shape[0]
    img_width = img.shape[1]
    
    for i in range(len(traces)):
        
        coor = traces[i]
        
        real_x = coor[0]
        real_y = coor[1]
        
        corrected_x =  int((img_width/real_width) * real_x) # the points are adjusted by a factor to match the cv2 format
        corrected_y =  int(img_height - (img_height/real_height) * real_y)
    
        
        img = cv2.circle(img, (corrected_x, corrected_y), color = color, radius = 1, thickness = 4) #drawing the points
        
        if i != 0 and arrow:
            
            img = cv2.arrowedLine(img, previous, (corrected_x, corrected_y), color = (0,255,0), thickness = 1)
        previous = (corrected_x, corrected_y)
    
    return img

In [ ]:
def extract_accelerometer_data(fname):
    df = read_traces_file(fname)
    acce_df = df[df[1]== "TYPE_ACCELEROMETER"]
    acce_df = acce_df.iloc[:,:5]
    time = acce_df[0].astype('int')
    x_acc = acce_df[2].astype('float')
    y_acc = acce_df[3].astype('float')
    z_acc = acce_df[4].astype('float')
    
    return time, x_acc, y_acc, z_acc
    
def plot_accelerometer(fname):
    
    t,x,y,z = extract_accelerometer_data(fname)
    _ = plt.figure(figsize = (15,15))
    plt.title("Acceleration Plot")
    plt.xlabel('Time')
    plt.ylabel('Acceleration')
    plt.plot(np.array(t), np.array(x), label = 'x-axis')
    plt.plot(np.array(t), np.array(y), label = 'y-axis')
    plt.plot(np.array(t), np.array(z), label = 'z-axis')
    plt.grid()
    plt.legend()
    plt.show()
    

In [ ]:
# showing examples 

image = cv2.imread("../input/indoor-location-navigation/metadata/5cd56b5ae2acfd2d33b58544/F1/floor_image.png")
meta_fname = "../input/indoor-location-navigation/metadata/5cd56b5ae2acfd2d33b58544/F1/floor_info.json"
trace_fname = "../input/indoor-location-navigation/train/5cd56b5ae2acfd2d33b58544/F1/5cf23c227427840009011eda.txt"

image = draw_map(image, meta_fname, trace_fname, arrow = True)
_ = plt.figure(figsize = (15,15))
plt.imshow(image)
plt.show()
plot_accelerometer(trace_fname)

image = cv2.imread("../input/indoor-location-navigation/metadata/5cd56b5ae2acfd2d33b58546/B1/floor_image.png")
meta_fname = "../input/indoor-location-navigation/metadata/5cd56b5ae2acfd2d33b58546/B1/floor_info.json"
trace_fname = "../input/indoor-location-navigation/train/5cd56b5ae2acfd2d33b58546/B1/5cf39cca5b96c60008d35add.txt"

image = draw_map(image, meta_fname, trace_fname, arrow = True)
_ = plt.figure(figsize = (15,15))
plt.imshow(image)
plt.show()
plot_accelerometer(trace_fname)

image = cv2.imread("../input/indoor-location-navigation/metadata/5cd56b5ae2acfd2d33b58548/1F/floor_image.png")
meta_fname = "../input/indoor-location-navigation/metadata/5cd56b5ae2acfd2d33b58548/1F/floor_info.json"
trace_fname = "../input/indoor-location-navigation/train/5cd56b5ae2acfd2d33b58548/1F/5cf20b12718b08000848a9f6.txt"

image = draw_map(image, meta_fname, trace_fname, arrow = True)
_ = plt.figure(figsize = (15,15))
plt.imshow(image)
plt.show()
plot_accelerometer(trace_fname)

In [ ]:
# Gyroscope data

def extract_gyroscope_data(fname):
    df = read_traces_file(fname)
    gyro_df = df[df[1]== "TYPE_GYROSCOPE"]
    gyro_df = gyro_df.iloc[:,:5]
    time = gyro_df[0].astype('int')
    x_gyro = gyro_df[2].astype('float')
    y_gyro = gyro_df[3].astype('float')
    z_gyro = gyro_df[4].astype('float')
    
    return time, x_gyro, y_gyro, z_gyro
    
def plot_accelerometer(fname):
    
    t,x,y,z = extract_gyroscope_data(fname)
    axis_name = ["x-axis", 'y-axis', 'z-axis']
    axis_data = [x,y,z]
    _ = plt.figure(figsize = (40,10))
    for i in range(3):
        
        plt.subplot(1,3, i+1)
        plt.xlabel('Time')
        plt.ylabel('Gyroscope Angle')
        plt.plot(np.array(t), np.array(axis_data[i]), label = axis_name[i])
        plt.grid()
        plt.legend()

plot_accelerometer("../input/indoor-location-navigation/train/5a0546857ecc773753327266/B1/5e15730aa280850006f3d005.txt")